In [1]:
import DTC.json_read_write as json_read_write
point_cloud = json_read_write.read_point_cloud_from_json("AllcityPC.json")
safe_areas = json_read_write.read_safe_areas_from_json("AllcitySA.json")
initialization_point = (116.20287663548845, 39.75112986803514)

In [2]:
import random
from DTC.noise_correction import NoiseCorrection
randomly_elected_trajectories = random.sample(point_cloud.trajectories, 1467)
noise_corrector = NoiseCorrection(safe_areas, initialization_point)

In [3]:
points_cleaned = 0
total_points = 0
for trajectory in randomly_elected_trajectories:
    _, labels_of_cleaned_points = noise_corrector.noise_detection(trajectory)
    points_cleaned += len(labels_of_cleaned_points)
    total_points += len(trajectory.points)
print("Points in randomly elected trajectories: ", total_points)
print("Points cleaned: ", points_cleaned)

Points in randomly elected trajectories:  49138
Points cleaned:  463


In [5]:
# use 5% of 52489 - or 5 % of all the points in the collective pointcloud for all trajectories
from math import ceil
amount_to_shift = ceil(total_points * 0.05)
print("Amount of points to shift: ", amount_to_shift)

list_of_points_in_randomly_elected_trajectories = []
for trajectory in randomly_elected_trajectories:
    list_of_points_in_randomly_elected_trajectories.extend(trajectory.points)

points_to_shift = random.sample(list_of_points_in_randomly_elected_trajectories, amount_to_shift)
print("Amount of points actually shifted: ", len(points_to_shift))

Amount of points to shift:  2457
Amount of points actually shifted:  2457


***We use random.sample to get a random set of trajectories every time this runs***
- furthermore, we take 5% of the points and shift them between 1-200m randomly with a random bearing

In [6]:
from DTC.distance_calculator import DistanceCalculator
for point in points_to_shift:
    point.set_coordinates(DistanceCalculator.shift_point_with_bearing(point, random.randint(1,200), random.randint(0,359)))
    point.noise = True

In [7]:
labels_of_cleaned_points = []
for trajectory in randomly_elected_trajectories:
    _, labels_of_locally_cleaned_points = noise_corrector.noise_detection(trajectory)
    labels_of_cleaned_points.extend(labels_of_locally_cleaned_points)
amount_of_cleaned_points = 0
amount_of_noisy_points_cleaned = 0
for label in labels_of_cleaned_points:
    amount_of_cleaned_points += 1
    if label == True:
        amount_of_noisy_points_cleaned += 1
print("Amount of cleaned points: ", amount_of_cleaned_points)
print("Amount of noisy points cleaned: ", amount_of_noisy_points_cleaned)
print("Amount of not noisy points cleaned: ", )
print("Amount of noisy points not cleaned: ", amount_to_shift - amount_of_noisy_points_cleaned)

243
Amount of cleaned points:  243
Amount of noisy points cleaned:  176
Amount of noisy points not cleaned:  2281


- After Cleaning, make comparison with the old trajectory points

**Test Setup**

- Using the setup from the paper DTC
- Using 1472 trajectories as in the paper in the first set of experiments
- 



**This is only with a MR with 2million points**

*first run*
- 926 cleaned
- 3107 to clean

*second run*
- 1140 cleaned
- 2564 to clean


**Running with 11 million points**

*first run*
- 739 cleaned
- 2573 to clean

*second run*
- 775 cleaned
- 2342 to clean

*third run*
- 850 cleaned
- 2394 to clean

*fourth run*
- 656 cleaned
- 2386 to clean

*fifth run*
- 1137 cleaned
- 3998 to clean

- VERY low amount of true positives - thats bad


In [ ]:
average_cleaned = sum([739, 775, 850, 656]) / sum([2573, 2342, 2394, 2386])
print(average_cleaned)

In [ ]:
from pyproj import Proj, transform

def setup_grid_to_geo_transformer(origin_latitude, origin_longitude):
    # Create a Proj instance for the local grid system. This could be UTM or any other fitting your local context.
    # Here, I'm using a simple Transverse Mercator projection centered on the origin with a specific false easting and northing.
    grid_proj = Proj(proj='tmerc', lat_0=origin_latitude, lon_0=origin_longitude, k=1, x_0=0, y_0=0, ellps='WGS84', units='m')

    # WGS84 Proj instance for lat/long
    geo_proj = Proj(proj='latlong', datum='WGS84')

    return grid_proj, geo_proj

def convert_grid_to_geo(x, y, grid_proj, geo_proj):
    # Transform from grid coordinates to geographic coordinates
    lon, lat = transform(grid_proj, geo_proj, x, y)
    return lat, lon

# Example usage
origin_latitude = 39.915  # Adjust to your grid's origin latitude
origin_longitude = 116.404  # Adjust to your grid's origin longitude
grid_proj, geo_proj = setup_grid_to_geo_transformer(origin_latitude, origin_longitude)

# Example grid coordinates
x, y = 100, 200  # These should be in meters if your grid system uses meters
lat, lon = convert_grid_to_geo(x, y, grid_proj, geo_proj)
print(f"Latitude: {lat}, Longitude: {lon}")